In [17]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

# Use your actual dataset path
DATA_DIR = r"C:\Users\Shriya\OneDrive\Documents\PRODIGY_ML_TASK_04\leapGestRecog"
IMG_SIZE = 64
GESTURE_LABELS = sorted(os.listdir(DATA_DIR))

def load_data():
    X, y = [], []
    for label_idx, gesture in enumerate(GESTURE_LABELS):
        gesture_path = os.path.join(DATA_DIR, gesture)
        for person in os.listdir(gesture_path):
            person_path = os.path.join(gesture_path, person)
            for img_file in os.listdir(person_path):
                img_path = os.path.join(person_path, img_file)
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                if img is None:
                    continue
                img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
                X.append(img)
                y.append(label_idx)
    return np.array(X), np.array(y)

def preprocess_data():
    X, y = load_data()
    X = X.astype('float32') / 255.0
    X = np.expand_dims(X, axis=-1)
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y)
    return X_train, X_val, y_train, y_val, GESTURE_LABELS


In [19]:
from tensorflow.keras import models, layers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

X_train, X_val, y_train, y_val, labels = preprocess_data()

y_train = to_categorical(y_train, num_classes=len(labels))
y_val = to_categorical(y_val, num_classes=len(labels))

model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(64,64,1)),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(len(labels), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
early_stop = EarlyStopping(monitor='val_loss', patience=5)

model.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val), callbacks=[early_stop])

model.save('gesture_model.h5')


c:\Users\Shriya\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 41s 73ms/step - accuracy: 0.7413 - loss: 0.7672 - val_accuracy: 0.9895 - val_loss: 0.0319
Epoch 2/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 36s 71ms/step - accuracy: 0.9910 - loss: 0.0303 - val_accuracy: 0.9908 - val_loss: 0.0170
Epoch 3/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 36s 71ms/step - accuracy: 0.9909 - loss: 0.0201 - val_accuracy: 0.9920 - val_loss: 0.0144
Epoch 4/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 36s 71ms/step - accuracy: 0.9919 - loss: 0.0168 - val_accuracy: 0.9927 - val_loss: 0.0142
Epoch 5/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 36s 72ms/step - accuracy: 0.9911 - loss: 0.0158 - val_accuracy: 0.9927 - val_loss: 0.0128
Epoch 6/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 36s 72ms/step - accuracy: 0.9932 - loss: 0.0133 - val_accuracy: 0.9902 - val_loss: 0.0158
Epoch 7/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 36s 72ms/step - accuracy: 0.9918 - loss: 0.0131 - val_accuracy: 0.9940 - val_loss: 0.0117
Epoch 8/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 36s 72ms/step - accuracy: 0.9934 - loss: 0.0119 - 